In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# Geometric Properties of Surfaces
todo

## Tangent Vectors

In [ ]:
"""
Tangent Vectors by Nearest Neighbors
Returns the evaluated Point and Vecs oriented toward the nearest neighbors along the u and v 
directions of a Surface.
"""
def nearest_neighbors(srf,u,v,include_negs = False):
    pt = Point(srf.func(u,v))
    
    du, dv = srf.tol_u/100.0, srf.tol_v/100.0
    vec_u = Vec( pt, srf.func( u+du, v ) )
    vec_v = Vec( pt, srf.func( u, v+dv ) )
    return pt, vec_u, vec_v

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P18.jpg" style="width: 200px; display: inline;">

In [ ]:
"""
Tangent Plane
Evaluates a Surface and returns the tangent Plane associated with the location determined by the 
given u and v parameters.
"""
def tangent_plane(srf,u,v):
    pt, vec_u, vec_v = srf.nearest_neighbors(u,v)
    return Plane(pt, vec_u.cross(vec_v) )

### Surface Area

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P19.jpg" style="width: 600px; display: inline;">

### Normal Vector

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P20.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P21.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P22.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P23.jpg" style="width: 200px; display: inline;">

In [ ]:
"""
Discrete Surface Unit Normal
Given a Mesh ngbrs that describes the triangular faces that surround a Surface patch, this routine 
calculates the weighted sum of the face normals of the mesh in order to approximate the surface unit 
normal.
"""
normal = Vec(0,0,0)
for face_idx in range(len(ngbrs.faces)):
    pts = ngbrs.face_pts(face_idx)
    face_area = 0.5*Vec(pts[0],pts[1]).cross(Vec(pts[0],pts[2])).length 
    normal = normal + ngbrs.face_normal(face_idx)*face_area

normal = normal.normalized()

## Curvature

### The Osculating Paraboloid

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P30.jpg" style="width: 600px; display: inline;">

### The Normal Curvatures

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P24.jpg" style="width: 200px; display: inline;">

<table style="width:600px">
    <tr>
        <th colspan="2" style="text-align:left">*Basic Surface Properties*</th>
    </tr>
    <tr>
        <td style="width:20%">Elliptic</td>
        <td style="width:80%">An elliptic surface point has principal curvatures which have the same sign and are different from zero. This is sometimes called double-curvature in the same direction.</td>
    </tr>
    <tr>
        <td style="width:20%">Umbilic</td>
        <td style="width:80%">An umbilic surface point is a special case of an elliptic surface point, in which the principal curvatures are the same. In such cases, since the maximum and minimum coincide, all normal curvatures must be the same, and thus the surface bends the same amount in all directions about the surface point.</td>
    </tr>
    <tr>
        <td style="width:20%">Hyperbolic</td>
        <td style="width:80%">A hyperbolic surface point has principal curvatures with opposite signs. This can be called double-curvature in opposite directions.</td>
    </tr>
    <tr>
        <td style="width:20%">Parabolic</td>
        <td style="width:80%">If one principal curvature is zero and the other nonzero, we call the point parabolic, or say that it manifests single-curvature.</td>
    </tr>
    <tr>
        <td style="width:20%">Flat</td>
        <td style="width:80%">For a flat surface point, both principal curvatures are zero. Since it follows that all normal curvatures are zero, the osculating paraboloid degenerates to the tangent plane, and there is no bending.</td>
    </tr>
</table>

#### Elliptic
An elliptic surface point has principal curvatures which have the same sign and are different from zero. This is sometimes called double-curvature in the same direction.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P25.jpg" style="width: 200px; display: inline;">

#### Umbilic
An umbilic surface point is a special case of an elliptic surface point, in which the principal curvatures are the same. In such cases, since the maximum and minimum coincide, all normal curvatures must be the same, and thus the surface bends the same amount in all directions about the surface point.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P26.jpg" style="width: 200px; display: inline;">

#### Hyperbolic
A hyperbolic surface point has principal curvatures with opposite signs. This can be called double-curvature in opposite directions.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P27.jpg" style="width: 200px; display: inline;">

#### Parabolic
If one principal curvature is zero and the other nonzero, we call the point parabolic, or say that it manifests single-curvature.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P28.jpg" style="width: 200px; display: inline;">

#### Flat
For a flat surface point, both principal curvatures are zero. Since it follows that all normal curvatures are zero, the osculating paraboloid degenerates to the tangent plane, and there is no bending.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.12.P29.jpg" style="width: 200px; display: inline;">

### Mean and Gaussian Curvature

In [ ]:
"""
Gaussian Curvature Heat Map
"""
msh = surf.to_mesh(divs_u=res_u,divs_v=res_v)
u_val = Interval(surf.u0, surf.u1).divide(res_u,True)
v_val = Interval(surf.v0, surf.v1).divide(res_v,True)

gaussian_curv = [surf.deval_curv(u,v,True)[3] for v in v_val for u in u_val]

colorA, colorB, colorC = Color(.97,.97,.97), Color(1, 0, 0), Color(0, 0, 1)
min_curv, max_curv = min(gaussian_curv), max(gaussian_curv)
denom = max(abs(min(gaussian_curv)), abs(max(gaussian_curv)))

meshes = Mesh.explode(msh)
for i, mesh in enumerate(meshes):
    if gaussian_curv[i] > 0: 
        mesh.set_color(Color.interpolate(colorA,colorB,gaussian_curv[i]/denom))
    else: 
        mesh.set_color(Color.interpolate(colorA,colorC,-gaussian_curv[i]/denom))